# Rankings

In [ ]:
ENV="dev"

## Connection to PostgreSQL database

In [ ]:
from itables import init_notebook_mode, show
import numpy as np
import pandas as pd

from postgres.PostgresToDataFrame import PostgresToDataframe

init_notebook_mode(all_interactive=True)

In [ ]:
db = PostgresToDataframe(ENV)

In [ ]:
def open_sql_file(path):
    with open(path, 'r') as sql_file:
        return sql_file.read()

## Rankings

### Sub functions

In [ ]:
db.execute_query(open_sql_file("sql/rankings/sub_functions.sql"))

### Teams rankings

In [ ]:
ID_CHP = 'ligue_1'
SEASON = '2023-2024'
SIDE = 'both'

N = 1000000
ROUND = 2

#### Expected Points (xP)

In [ ]:
# def game_sim(home: pd.Series, away: pd.Series) -> tuple:
#     home_xgps: float = home['xg'] / home['shots']  # xG per shot
#     away_xgps: float = away['xg'] / away['shots']

#     home_xg = np.random.binomial(home['shots'], home_xgps)  # simulates whether or not each shot leads to a goal
#     away_xg = np.random.binomial(away['shots'], away_xgps)

#     if home_xg > away_xg:
#         home_pts, away_pts = 3,0  
#     elif home_xg == away_xg:
#         home_pts, away_pts = 1,1 
#     else:
#         home_pts, away_pts = 0,3

#     return home_pts, away_pts

# def build_justice_table(teams: pd.DataFrame, n: int) -> pd.DataFrame:
#     home_res = []
#     away_res = []

#     home: pd.Series = teams.loc[teams['played_home'] == True, ['shots', 'xg']]
#     away: pd.Series = teams.loc[teams['played_home'] == False, ['shots', 'xg']]

#     for _ in range(n):
#         sim_res = game_sim(home, away)
#         home_res.append(sim_res[0])
#         away_res.append(sim_res[1])

#     # Utilisez .loc pour mettre à jour la colonne 'xp' spécifique
#     teams.loc[teams['played_home'] == True, 'xp'] = round(sum(home_res) / n, 3)
#     teams.loc[teams['played_home'] == False, 'xp'] = round(sum(away_res) / n, 3)

#     return teams[['id_match', 'played_home', 'complete_name', 'xp']]

# justice_table = pd.DataFrame()

# for match in team_stats['id_match'].unique():
#     justice_table = pd.concat([justice_table, build_justice_table(team_stats.loc[team_stats['id_match'] == match].copy(), 10000)])

# justice_table

In [ ]:
def simulate_matches(teams: pd.DataFrame, n: int) -> pd.DataFrame:
    home_teams = teams.loc[teams['played_home'] == True, ['shots', 'xg']]
    away_teams = teams.loc[teams['played_home'] == False, ['shots', 'xg']]

    # Utilisation d'opérations vectorisées pour la simulation des matchs
    home_goals = np.random.binomial(home_teams['shots'], home_teams['xg'] / home_teams['shots'], size=(n,))
    away_goals = np.random.binomial(away_teams['shots'], away_teams['xg'] / away_teams['shots'], size=(n,))

    # Attribution des points en fonction du résultat du match
    home_points = np.where(home_goals > away_goals, 3, np.where(home_goals == away_goals, 1, 0))
    away_points = np.where(home_points == 1, 1, 3-home_points)

    # Mise à jour de la colonne 'xp' pour chaque équipe
    teams.loc[teams['played_home'] == True, 'xP'] = round(np.mean(home_points), ROUND)
    teams.loc[teams['played_home'] == False, 'xP'] = round(np.mean(away_points), ROUND)

    return teams[['id_match', 'played_home', 'Club', 'xP']]


def build_justice_ranking(team_stats: pd.DataFrame, side: str = 'both') -> pd.DataFrame:
    if side == 'home':
        query = 'played_home'
    elif side == 'away':
        query = 'not played_home'
    elif side == 'both':
        query = '(played_home or not played_home)'
        
    justice_table = pd.concat([simulate_matches(team_stats.loc[team_stats['id_match'] == id_match].copy(), N) for id_match in team_stats['id_match'].unique()])
    
    return justice_table.query(query)[['Club', 'xP']].groupby('Club').sum('xP')

In [ ]:
team_stats = db.df_from_query(f"""
                              select ts.id_match, c.complete_name as "Club", ts.played_home, ts.xg, ts.shots 
                              from team_stats ts 
                              join club c on ts.id_team = c.id 
                              join match m on m.id = ts.id_match 
                              where m.id_championship = '{ID_CHP}' and m.season = '{SEASON}';
                              """)
justice_ranking = build_justice_ranking(team_stats)

#### Basic Ranking

In [ ]:
db.execute_query(open_sql_file("sql/rankings/teams.sql"))

In [ ]:
teams_ranking = db.df_from_query(f"""select * from teams_rankings(id_chp := '{ID_CHP}', id_season := '{SEASON}', side := '{SIDE}');""").set_index("Ranking")


#### Merged Ranking

In [ ]:
teams_ranking = pd.merge(teams_ranking, justice_ranking, left_on='Club', right_index=True)
teams_ranking['Diff Points'] = teams_ranking['Points'] - teams_ranking['xP']

In [ ]:
show(teams_ranking[['Matches', 'Club', 'Wins', 'Draws', 'Loses', 'Points', 'xP', 'Diff Points']].set_index('Club').loc['FC Lorient'], paging=False)

### Players Rankings

In [ ]:
db.execute_query(open_sql_file("sql/rankings/players.sql"))

In [ ]:
players_ranking = db.df_from_query("select * from players_rankings(id_chp := 'ligue_1', id_season := '2023-2024', side := 'both');").set_index("Ranking")

In [ ]:
players_ranking

## Distinct positions

In [ ]:
all_positions = db.df_from_query("select position from player_stats ps group by position;")["position"].dropna().tolist()
flattened_positions = []
for sublist in [position.split(',') for position in all_positions]:
    flattened_positions.extend(sublist)

In [ ]:
list(set(all_positions))

In [ ]:
unique_positions = list(set(flattened_positions))
general_positions = ['gk', 'def', 'mid', 'att']

{
    'gk': 'gk',
    'df': 'def',
    'lb': 'def',
    'cb': 'def',
    'wb': 'def', #wide back
    'rb': 'def',
    'dm': 'mid',
    'cm': 'mid',
    'lm': 'mid',
    'rm': 'mid',
    'mf': 'mid', #in-sub players
    'am': 'mid', #attacking midfielder
    'lw': 'att', #winger
    'rw': 'att',
    'fw': 'att'
}

# Closing the PostgreSQL session

In [ ]:
db.close()